In [ ]:
# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
# from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit_ibm_runtime import Estimator, Sampler, Session, Options

# SciPy minimizer routine
from scipy.optimize import minimize

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw

from typing import Tuple, Dict

In [ ]:
# The edge syntax is (start, end, weight)
GRAPH_NAME = "MAX_CUT"
edges = [(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 0, 1)]
num_nodes = 4

In [ ]:
G = rx.PyGraph()
G.add_nodes_from(range(num_nodes))
G.add_edges_from(edges)
mpl_draw(G, pos=rx.bipartite_layout(G, {0}), with_labels=True, node_color="#EE5396", font_color="#F4F4F4")

In [ ]:
hamiltonian = SparsePauliOp.from_list([("IIZZ", 1), ("IZZI", 1), ("ZZII", 1), ("ZIIZ", 1)])
ansatz = QAOAAnsatz(hamiltonian, reps=2)
ansatz.decompose(reps=3).draw(output="mpl", style="iqp")

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [ ]:
from qiskit.primitives import Estimator, Sampler

estimator = Estimator(options = {"shots": 1024})
sampler = Sampler(options = {"shots": 1024})

In [ ]:
x0 = 2 * np.pi * np.random.rand(ansatz.num_parameters)
x0

In [ ]:
res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
res

In [ ]:
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_distribution = sampler.run(qc).result().quasi_dists[0]
plot_distribution(samp_distribution.binary_probabilities())

In [ ]:
# Statistics
def max_global_statistics(num_layers=1, num_generations=100) -> Tuple[Dict[str, float],
                                                                      Dict[str, float]]:
    max_statistics = {}
    global_statistics = {}
    ansatz = QAOAAnsatz(hamiltonian, reps=num_layers)
    x0 = [1.0, 1.0] * num_layers
    for iteration in range(0, num_generations):
        theta_res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
        qc = ansatz.assign_parameters(theta_res.x)
        qc.measure_all()
        counts = sampler.run(qc).result().quasi_dists[0].binary_probabilities()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)
    return max_statistics, global_statistics

print(GRAPH_NAME + ":\n")
interval_num_layers = (1, 5)
for p in range(interval_num_layers[0], interval_num_layers[1] + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=100)
    print(f"Max statistics (p = {p}): ", str(max_st))
    print(f"Global statistics (p = {p}): ", str(global_st))
    print()

In [ ]:
!dunstify -t $((12 * 60*60*1000)) --urgency=critical "Ejecución completada"